In [1]:
## IMPORTS

import os
import sys
import time
import math
from datetime import datetime
from io import StringIO


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from numpy.polynomial.polynomial import Polynomial
import scipy.integrate as integrate
import scipy.optimize as sco
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import openpyxl
import networkx as nx
import sklearn.metrics
from IPython.core.display import HTML
import plotly.io as pio
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl as opxl
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import time


sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss\ansys')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model')
import jou_gen as jg
import ansys_py_bridge as apb
import ansys.fluent.core as pyfluent
import model as ty
import proc as pr
import plot_functions_here as pfun
import heat_transfer as bht
import fluids as fds
import ht
import general as gen

In [ ]:
## Importation rapide

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI'

report_path = folder_path

panelSpecs = pr.import_geometry(geometry_path)
hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

## Initialisation

# File > Application > Start > folder_path_ansys
# Ensuite copier le numéro du server et le modifier ci-dessous
solver_path = folder_path_ansys + '\\server\\' + 'server_info-25532.txt'
# Créer une session
solver = pyfluent.connect_to_fluent(server_info_file_name=solver_path)
tui = solver.tui

In [ ]:
## Notebook qui fait la boucle sur les cas

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
# nb_case = len(stepConditions)
nb_case = 5




for case in range(nb_case) :
# case = 7
# if True :

    ## Début de la boucle for case

    hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
    big_it = hyp['big_it']
    T_amb = stepConditions[case]['T_amb']
    T_fluid_in0 = stepConditions[case]['T_fluid_in0']
    # save_stepConditions = stepConditions[case].copy() ## Modification directement dans steady_one_steady_state_all_he

    # Créer le chemin complet du dossier de résultats
    folder_path_case = os.path.join(folder_path, f"cas_{case}")

    # Vérifier si le dossier n'existe pas déjà
    if not os.path.exists(folder_path_case):
        # Créer le dossier s'il n'existe pas
        os.makedirs(folder_path_case)

    hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')

    file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
    file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
    file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
    file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'

    Elapsed_time = pd.DataFrame(columns=['iteration', 'time'])

    # File > Application > Start > folder_path_ansys
    # Ensuite copier le numéro du server et le modifier ci-dessous
    solver_path = folder_path_ansys + '\server\\' + 'server_info-27608.txt'
    # Créer une session
    solver = pyfluent.connect_to_fluent(server_info_file_name=solver_path)
    tui = solver.tui




    ## Modification des paramètres invariables

    # On modifie les composantes de la gravité en fonction de l'inclinaison du panneau
    theta = apb.get_value('theta', 'named_expression', Inputs_PyFluent)
    jg.change_gravity(tui, theta)

    # On initialise la température ambiante
    jg.change_named_expression(tui, 'T_amb', T_amb, 'K')

    # On initialise le critère de convergence pour les transferts thermiques
    convergence_criteria = apb.get_value('convergence_criteria', 'named_expression', Inputs_PyFluent)
    jg.change_named_expression(tui, 'convergence_criteria', convergence_criteria, 'W')




    ## Première initialisation des paramètres dans ANSYS Fluent

    for i in range(len(Inputs_PyFluent)):
        named_expression = Inputs_PyFluent.iloc[i]['named_expression']
        value = Inputs_PyFluent.iloc[i]['value']
        unit = Inputs_PyFluent.iloc[i]['unit']
        jg.change_named_expression(tui, named_expression, value, unit)




    # Initialisation des valeurs avec PVT-thermal-performances-model
    df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)
    # stepConditions[case] = save_stepConditions.copy()

    # On initialise la première température avec le modèle et le transfert radiatif
    apb.set_value('T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in0)


    e_PV = 2.5e-3
    volume_PV = 1134*1708*e_PV*1e-6
    Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
    apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

    # On initialise les a_f et b_f avec le modèle
    for i in range(1, nb_hx + 1):
        apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
        apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])
    

    # Met à jour les méthodes dans les hypothèses
    hyp['method_h_top_g_exchanger'] = 'CFD'
    hyp['method_h_back_abs'] = 'CFD'
    hyp['method_h_back_tube'] = 'CFD'
    hyp['method_h_rad_back_tube'] = 'CFD'



    ## Initialisation
    L_abs = panelSpecs['main']['L_abs']
    c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
    jg.standard_initialization(tui, 'cd_fc', 0, 0, 0, c)


    limit_big_it = 7 # Nombre d'itérations maximum
    convergence_criteria = 1e-1 # Critère de convergence pour le transfert thermique
    nb_it = 30 # Nombre d'itération lancée sur ANSYS
    iteration = 0


    while iteration < limit_big_it :
        
        ## Début boucle itération

        time_start = time.time()

        # try : 
        ## Calcul des températures

        # apb.fill_initial_temperature_dict(Inputs_PyFluent)

        for i in range(2, nb_hx+2) :
            T_fluid_in = res[f'part{i}']['df_one']['T_fluid_in'].loc[0]
            apb.set_value(f'T_fluid_in_{i-1}', 'named_expression', Inputs_PyFluent, T_fluid_in)
        T_fluid_in_man = res[f'part1']['df_one']['T_fluid_mean'].loc[0]
        T_fluid_out_man = res[f'part7']['df_one']['T_fluid_mean'].loc[0]
        apb.set_value(f'T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in_man)
        apb.set_value(f'T_fluid_out_man', 'named_expression', Inputs_PyFluent, T_fluid_out_man)


        ## Sauvegarde des données du modèle 1D
        # Sauvegarde de df_one
        keys = list(res['part1']['df_one'].keys())
        values_dict = {key: [] for key in keys}

        # Collecte des valeurs dans des listes pour chaque clé
        for i in range(1, nb_hx+3):
            part_key = f'part{i}'
            if part_key in res:
                for key in keys:
                    values_dict[key].append(res[part_key]['df_one'][key].values[0])

        # Création du DataFrame à partir des listes de valeurs
        result_1D_df_one = pd.DataFrame(values_dict)

        # Ajouter les noms des index pour refléter les parties
        result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
        result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

        # Sauvegarde de slices_df
        keys = list(res['part1']['slices_df'].keys())
        values_dict = {key: [] for key in keys}

        # Collecte des valeurs dans des listes pour chaque clé
        for part in range(1, nb_hx+3):
            part_key = f'part{part}'
            if part_key in res:
                for key in keys:
                    values_dict[key].append(res[part_key]['slices_df'][key].values[0])

        # Création du DataFrame à partir des listes de valeurs
        result_1D_slices_df = pd.DataFrame(values_dict)
        result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
        result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)





        ## Modification des paramètres dans ANSYS Fluent
        apb.set_value('T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in0) # Au cas où
        for part in range(1, nb_hx+1):
            T_fluid_in = apb.get_value(f'T_fluid_in_{part}', 'named_expression', Inputs_PyFluent)
            a_f = apb.get_value(f'a_f_{part}', 'named_expression', Inputs_PyFluent)
            b_f = apb.get_value(f'b_f_{part}', 'named_expression', Inputs_PyFluent)
            jg.change_named_expression(tui, f'T_fluid_in_{part}', T_fluid_in, 'K')
            jg.change_named_expression(tui, f'a_f_{part}', a_f, 'm^-1')
            jg.change_named_expression(tui, f'b_f_{part}', b_f, 'K m^-1')
        jg.change_named_expression(tui, 'Heat_Generation_Rate', Heat_Generation_Rate, 'W/m^3')
        jg.change_named_expression(tui, 'T_fluid_in_man',T_fluid_in_man, 'K')
        jg.change_named_expression(tui, 'T_fluid_out_man', T_fluid_out_man, 'K')



        ## Lancement de la simulation

        solver.solution.run_calculation.iterate(number_of_iterations=nb_it)




        jg.write_report(tui,'ht',folder_path_case,f'ht_tot_report_{big_it}')
        jg.write_report(tui,'rad_ht',folder_path_case,f'ht_rad_report_{big_it}')

        Qdot_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{big_it}.csv'))
        Qdot_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{big_it}.csv'))

        Qdot_conv = pd.DataFrame(columns=['Component', 'conv_ht'])
        Qdot_conv['Component'] = Qdot_tot['Component']
        Qdot_conv['conv_ht'] = Qdot_tot['ht']-Qdot_rad['rad_ht']

        Qdot_conv.to_csv(os.path.join(folder_path_case,f'ht_conv_report_{big_it}.csv'),sep=';', index=False)

        hx_list =  ['hx_bend_mid_air' , 'hx_bend_mid_pv' , 'hx_bend_yd_air' , 'hx_bend_yd_pv' , 'hx_bend_yu_air' ,
            'hx_bend_yu_pv' , 'hx_flat_yd_air', 'hx_flat_yu_air', 'manifold_yu', 'manifold_yd']

        ## Partie très moche, difficile de faire autrement pour le moment
        part1 = ['manifold_yu']
        part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
        part3 = ['hx_flat_yu_air']
        part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
        part5 = ['hx_flat_yd_air']
        part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
        part7 = ['manifold_yd']

        part1_top = []
        part2_top = []
        part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
        part4_top = []
        part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
        part6_top = []
        part7_top = []


        Areas_top = []
        Areas_back = []
        for part in panelSpecs['decomp'].keys() :
            if part != 'main':
                Areas_back.append(pr.back_area_tube_conv_and_rad(panelSpecs[part])/4.75)
                Areas_top.append(pr.top_area_tube_contact_PV(panelSpecs[part])/4.75)


        ## Sauvegarde des données du modèle CFD
        phi_top_list = []
        phi_tube_list = []
        phi_abs_list = []

        for i in range(1, nb_hx+3) :
            if i == 3 or i == 5 :
                phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                phi_top_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum()/Areas_top[i-1])
                phi_abs_list.append(1e-6)
            else :
                phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                phi_top_list.append(1e-6)
                phi_abs_list.append(1e-6) # Dans le modèle, on a fait en sorte de ne pas avoir besoin de cette valeur

        result_CFD = pd.DataFrame(columns=['component', 'phi_top', 'phi_abs', 'phi_tube'])
        result_CFD['component'] = [f'part{i}' for i in range(1, nb_hx+3)]
        result_CFD['phi_top'] = phi_top_list 
        result_CFD['phi_abs'] = phi_abs_list
        result_CFD['phi_tube'] = phi_tube_list 



        ## Envoi des flux au PVT-thermal-performance-model
        result_CFD.to_csv(file_path_result_CFD,sep=';', index=False)
        Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)


        # Pour permettre au modèle de ne pas recommencer depuis le début
        hyp['specific_init'] = df_one['T_PV'].values[0]

        ## Appel du PVT-thermal-performances-model
        df_one,res = ty.simu_one_steady_state_all_he(panelSpecs,stepConditions[case],hyp)
        # stepConditions[case] = save_stepConditions.copy() ## On remet les conditions initiales car simu_one_steady_state_all_he modifie stepConditions

        Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
        apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

        # On initialise les a_f et b_f avec le modèle
        for i in range(1, nb_hx + 1):
            apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
            apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])



        ## On incrémente 

        big_it = big_it + 1
        hyp['big_it'] = big_it
        file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
        file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
        file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
        file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'


        print(f'Cas {case} iteration {big_it} / {limit_big_it} terminée')
        iteration += 1

        # except Exception as e:
        #     big_it = big_it + 1
        #     print(f'Cas {case} iteration {big_it} / {limit_big_it} non aboutie : {e}')
        #     iteration = limit_big_it
        #     continue

        jg.write_data(tui, folder_path_case, f'cas{case}_it{big_it}')
        jg.write_residuals_file(tui, folder_path_case, f'residuals_cas{case}_it{big_it}')
        jg.write_case(tui, folder_path_case, f'cas{case}_it{big_it}')

        time_end = time.time()
        Elapsed_time.loc[iteration] = {'iteration': big_it, 'time': time_end - time_start}

        # ## Fin boucle itération

    Elapsed_time.to_csv(os.path.join(folder_path_case,f'Elapsed_time_cas_{case}.csv'),sep=';', index=False)

## Fin de la boucle for case


## Dernière sauvegarde des données du modèle 1D
# Sauvegarde de df_one
keys = list(res['part1']['df_one'].keys())
values_dict = {key: [] for key in keys}

# Collecte des valeurs dans des listes pour chaque clé
for i in range(1, nb_hx+3):
    part_key = f'part{i}'
    if part_key in res:
        for key in keys:
            values_dict[key].append(res[part_key]['df_one'][key].values[0])

# Création du DataFrame à partir des listes de valeurs
result_1D_df_one = pd.DataFrame(values_dict)

# Ajouter les noms des index pour refléter les parties
result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

# Sauvegarde de slices_df
keys = list(res['part1']['slices_df'].keys())
values_dict = {key: [] for key in keys}

# Collecte des valeurs dans des listes pour chaque clé
for part in range(1, nb_hx+3):
    part_key = f'part{part}'
    if part_key in res:
        for key in keys:
            values_dict[key].append(res[part_key]['slices_df'][key].values[0])

# Création du DataFrame à partir des listes de valeurs
result_1D_slices_df = pd.DataFrame(values_dict)
result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)


In [ ]:
## Calcul des différents cas avec simplement le modèle 1D

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
nb_case = 5


hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

for case in range(nb_case) :
    big_it = hyp['big_it']
    T_amb = stepConditions[case]['T_amb']
    T_fluid_in0 = stepConditions[case]['T_fluid_in0']
    # save_stepConditions = stepConditions[case].copy() ## Modification directement dans steady_one_steady_state_all_he

    # Créer le chemin complet du dossier de résultats
    folder_path_case = os.path.join(folder_path, f"cas_{case}")

    # Vérifier si le dossier n'existe pas déjà
    if not os.path.exists(folder_path_case):
        # Créer le dossier s'il n'existe pas
        os.makedirs(folder_path_case)

    hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test') ## Nomeclature pour la comparaison des résultat

    file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
    file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
    file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'


    # Initialisation des valeurs avec PVT-thermal-performances-model
    df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)


    ## Calcul des températures
    for i in range(2, nb_hx+2) :
        T_fluid_in = res[f'part{i}']['df_one']['T_fluid_in'].loc[0]
        apb.set_value(f'T_fluid_in_{i-1}', 'named_expression', Inputs_PyFluent, T_fluid_in)
    T_fluid_in_man = res[f'part1']['df_one']['T_fluid_mean'].loc[0]
    T_fluid_out_man = res[f'part7']['df_one']['T_fluid_mean'].loc[0]
    apb.set_value(f'T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in_man)
    apb.set_value(f'T_fluid_out_man', 'named_expression', Inputs_PyFluent, T_fluid_out_man)


    ## Sauvegarde des données du modèle 1D
    # Sauvegarde de df_one
    keys = list(res['part1']['df_one'].keys())
    values_dict = {key: [] for key in keys}

    # Collecte des valeurs dans des listes pour chaque clé
    for i in range(1, nb_hx+3):
        part_key = f'part{i}'
        if part_key in res:
            for key in keys:
                values_dict[key].append(res[part_key]['df_one'][key].values[0])

    # Création du DataFrame à partir des listes de valeurs
    result_1D_df_one = pd.DataFrame(values_dict)

    # Ajouter les noms des index pour refléter les parties
    result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
    result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

    # Sauvegarde de slices_df
    keys = list(res['part1']['slices_df'].keys())
    values_dict = {key: [] for key in keys}

    # Collecte des valeurs dans des listes pour chaque clé
    for part in range(1, nb_case+3):
        part_key = f'part{part}'
        if part_key in res:
            for key in keys:
                values_dict[key].append(res[part_key]['slices_df'][key].values[0])

    # Création du DataFrame à partir des listes de valeurs
    result_1D_slices_df = pd.DataFrame(values_dict)
    result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
    result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)

    Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)



In [4]:
## Calcul du cas de référence avec différents maillages en uniforme 

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))

case = 7

hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
big_it = hyp['big_it']
T_amb = stepConditions[case]['T_amb']
T_fluid_in0 = stepConditions[case]['T_fluid_in0']

# File > Application > Start > folder_path_ansys
# Ensuite copier le numéro du server et le modifier ci-dessous
solver_path = folder_path_ansys + '\server\\' + 'server_info-37096.txt'
# Créer une session
solver = pyfluent.connect_to_fluent(server_info_file_name=solver_path)
tui = solver.tui


for geom in range(nb_geom) :

    geom_path = geometries_dict['mesh_path'].iloc[geom]
    geom_name = geometries_dict['mesh_name_wo_ext'].iloc[geom]
    geom_panel_name = geometries_dict['mesh_panel'].iloc[geom]

    folder_path_geom = os.path.join(folder_path, f'cas_M{geom}')
    # Vérifier si le dossier n'existe pas déjà
    if not os.path.exists(folder_path_geom):
        # Créer le dossier s'il n'existe pas
        os.makedirs(folder_path_geom)

    jg.change_mesh(tui, geom_path, geom_name)
    jg.create_radiation(tui, geom_name)

    folder_path_case = folder_path_geom
    hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test_') ## Nomeclature pour la comparaison des résultats

    file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
    file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
    file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
    file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'




    ## Modification des paramètres invariables

    # On modifie les composantes de la gravité en fonction de l'inclinaison du panneau
    theta = apb.get_value('theta', 'named_expression', Inputs_PyFluent)
    jg.change_gravity(tui, theta)

    # On initialise la température ambiante
    jg.change_named_expression(tui, 'T_amb', T_amb, 'K')

    # On initialise le critère de convergence pour les transferts thermiques
    convergence_criteria = apb.get_value('convergence_criteria', 'named_expression', Inputs_PyFluent)
    jg.change_named_expression(tui, 'convergence_criteria', convergence_criteria, 'W')



    ## Première initialisation des paramètres dans ANSYS Fluent

    for i in range(len(Inputs_PyFluent)):
        named_expression = Inputs_PyFluent.iloc[i]['named_expression']
        value = Inputs_PyFluent.iloc[i]['value']
        unit = Inputs_PyFluent.iloc[i]['unit']
        jg.change_named_expression(tui, named_expression, value, unit)



    # Initialisation des valeurs avec PVT-thermal-performances-model
    df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)
    # stepConditions[case] = save_stepConditions.copy()

    # On initialise la première température avec le modèle et le transfert radiatif
    apb.set_value('T_fluid_in_1', 'named_expression', Inputs_PyFluent, T_fluid_in0)


    e_PV = 2.5e-3
    volume_PV = 1134*1708*e_PV*1e-6
    Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
    apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

    # On initialise les a_f et b_f avec le modèle
    for i in range(1, nb_hx + 1):
        apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
        apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])



    ## Initialisation
    L_abs = panelSpecs['main']['L_abs']
    c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
    jg.standard_initialization(tui, 'cd_fc', 0, 0, 0, c)



    nb_it = 1000 # Nombre d'itération lancée sur ANSYS

    ## Lancement de la simulation

    solver.solution.run_calculation.iterate(number_of_iterations=nb_it)







    jg.write_report(tui,'ht',folder_path_case,f'ht_tot_report_{big_it}')
    jg.write_report(tui,'rad_ht',folder_path_case,f'ht_rad_report_{big_it}')

    Qdot_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{big_it}.csv'))
    Qdot_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{big_it}.csv'))

    Qdot_conv = pd.DataFrame(columns=['Component', 'conv_ht'])
    Qdot_conv['Component'] = Qdot_tot['Component']
    Qdot_conv['conv_ht'] = Qdot_tot['ht']-Qdot_rad['rad_ht']

    Qdot_conv.to_csv(os.path.join(folder_path_case,f'ht_conv_report_{big_it}.csv'),sep=';', index=False)

    hx_list =  ['hx_bend_mid_air' , 'hx_bend_mid_pv' , 'hx_bend_yd_air' , 'hx_bend_yd_pv' , 'hx_bend_yu_air' ,
        'hx_bend_yu_pv' , 'hx_flat_yd_air', 'hx_flat_yu_air', 'manifold_yu', 'manifold_yd']

    ## Partie très moche, difficile de faire autrement pour le moment
    part1 = ['manifold_yu']
    part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
    part3 = ['hx_flat_yu_air']
    part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
    part5 = ['hx_flat_yd_air']
    part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
    part7 = ['manifold_yd']

    part1_top = []
    part2_top = []
    part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
    part4_top = []
    part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
    part6_top = []
    part7_top = []


    Areas_top = []
    Areas_back = []
    for part in panelSpecs['decomp'].keys() :
        if part != 'main':
            Areas_back.append(pr.back_area_tube_conv_and_rad(panelSpecs[part])/4.75)
            Areas_top.append(pr.top_area_tube_contact_PV(panelSpecs[part])/4.75)


    ## Sauvegarde des données du modèle CFD
    phi_top_list = []
    phi_tube_list = []
    phi_abs_list = []

    for i in range(1, nb_hx+3) :
        if i == 3 or i == 5 :
            phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
            phi_top_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum()/Areas_top[i-1])
            phi_abs_list.append(1e-6)
        else :
            phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
            phi_top_list.append(1e-6)
            phi_abs_list.append(1e-6) # Dans le modèle, on a fait en sorte de ne pas avoir besoin de cette valeur

    result_CFD = pd.DataFrame(columns=['component', 'phi_top', 'phi_abs', 'phi_tube'])
    result_CFD['component'] = [f'part{i}' for i in range(1, nb_hx+3)]
    result_CFD['phi_top'] = phi_top_list 
    result_CFD['phi_abs'] = phi_abs_list
    result_CFD['phi_tube'] = phi_tube_list 



    ## Envoi des flux au PVT-thermal-performance-model
    result_CFD.to_csv(file_path_result_CFD,sep=';', index=False)
    Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)

    jg.write_data(tui, folder_path_case, f'cas_ref_CFD_uniform')
    jg.write_residuals_file(tui, folder_path_case, f'residuals_cas_ref_CFD_uniform')
    jg.write_case(tui, folder_path_case, f'cas_ref_CFD_uniform')


Reading journal file D:\ANSYS Fluent Projects\temp\change_mesh.txt...

> /file/set-tui-version "22.2"

> (cx-gui-do cx-activate-item "MenuBar*ReadSubMenu*Mesh...")

(cx-gui-do cx-set-toggle-button2 "Read Mesh Options*Table1(Options)*ToggleBox1*Replace mesh" #t)

(cx-gui-do cx-activate-item "Read Mesh Options*Table1(Options)*ToggleBox1*Replace mesh")

(cx-gui-do cx-activate-item "Read Mesh Options*PanelButtons*PushButton1(OK)")



> (cx-gui-do cx-set-file-dialog-entries "Select File" '( "D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\Mesh\V4.5NI_simplified_Vref_M1.msh.h5") "CFF Mesh Files (*.msh.h5 )")

Writing Settings file "C:\Users\seagu\AppData\Local\Temp\10"...
	writing rp variables ... Done.
	writing domain variables ... Done.
	writing frame-corps.1 (type solid) (mixture) ... Done.
	writing frame-corps (type solid) (mixture) ... Done.
	writing pv-corps (type solid) (mixture) ... Done.
	writing cd-cd2 (type fluid) (mixture) ... Done.
	writing cd-cd1 (type fluid) (mixture) .

NameError: name 'hyp' is not defined

### Cas de référence

In [ ]:
## Calcul du cas de référence avec simplement le modèle 1D

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
case = 7


hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
big_it = hyp['big_it']
T_amb = stepConditions[case]['T_amb']
T_fluid_in0 = stepConditions[case]['T_fluid_in0']
# save_stepConditions = stepConditions[case].copy() ## Modification directement dans steady_one_steady_state_all_he

# Créer le chemin complet du dossier de résultats
folder_path_case = os.path.join(folder_path, f"cas_ref")

# Vérifier si le dossier n'existe pas déjà
if not os.path.exists(folder_path_case):
    # Créer le dossier s'il n'existe pas
    os.makedirs(folder_path_case)

hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test_1D') ## Nomeclature pour la comparaison des résultats

file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'


# Initialisation des valeurs avec PVT-thermal-performances-model
df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)


## Calcul des températures
apb.fill_initial_temperature_dict(Inputs_PyFluent)


## Sauvegarde des données du modèle 1D
# Sauvegarde de df_one
keys = list(res['part1']['df_one'].keys())
values_dict = {key: [] for key in keys}

# Collecte des valeurs dans des listes pour chaque clé
for i in range(1, nb_hx+3):
    part_key = f'part{i}'
    if part_key in res:
        for key in keys:
            values_dict[key].append(res[part_key]['df_one'][key].values[0])

# Création du DataFrame à partir des listes de valeurs
result_1D_df_one = pd.DataFrame(values_dict)

# Ajouter les noms des index pour refléter les parties
result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

# Sauvegarde de slices_df
keys = list(res['part1']['slices_df'].keys())
values_dict = {key: [] for key in keys}

# Collecte des valeurs dans des listes pour chaque clé
for part in range(1, nb_case+3):
    part_key = f'part{part}'
    if part_key in res:
        for key in keys:
            values_dict[key].append(res[part_key]['slices_df'][key].values[0])

# Création du DataFrame à partir des listes de valeurs
result_1D_slices_df = pd.DataFrame(values_dict)
result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)

Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)



In [ ]:
## Calcul du cas de référence avec température uniforme sur hx en CFD

## Importation des données excel

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))

case = 7

hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
big_it = hyp['big_it']
T_amb = stepConditions[case]['T_amb']
T_fluid_in0 = stepConditions[case]['T_fluid_in0']

# Créer le chemin complet du dossier de résultats
folder_path_case = os.path.join(folder_path, f"cas_M3")

# Vérifier si le dossier n'existe pas déjà
if not os.path.exists(folder_path_case):
    # Créer le dossier s'il n'existe pas
    os.makedirs(folder_path_case)

hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test_CFD_uniform') ## Nomeclature pour la comparaison des résultats

file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'


# File > Application > Start > folder_path_ansys
# Ensuite copier le numéro du server et le modifier ci-dessous
solver_path = folder_path_ansys + '\server\\' + 'server_info-30284.txt'
# Créer une session
solver = pyfluent.connect_to_fluent(server_info_file_name=solver_path)
tui = solver.tui





## Modification des paramètres invariables

# On modifie les composantes de la gravité en fonction de l'inclinaison du panneau
theta = apb.get_value('theta', 'named_expression', Inputs_PyFluent)
jg.change_gravity(tui, theta)

# On initialise la température ambiante
jg.change_named_expression(tui, 'T_amb', T_amb, 'K')

# On initialise le critère de convergence pour les transferts thermiques
convergence_criteria = apb.get_value('convergence_criteria', 'named_expression', Inputs_PyFluent)
jg.change_named_expression(tui, 'convergence_criteria', convergence_criteria, 'W')



## Première initialisation des paramètres dans ANSYS Fluent

for i in range(len(Inputs_PyFluent)):
    named_expression = Inputs_PyFluent.iloc[i]['named_expression']
    value = Inputs_PyFluent.iloc[i]['value']
    unit = Inputs_PyFluent.iloc[i]['unit']
    jg.change_named_expression(tui, named_expression, value, unit)



# Initialisation des valeurs avec PVT-thermal-performances-model
df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)
# stepConditions[case] = save_stepConditions.copy()

# On initialise la première température avec le modèle et le transfert radiatif
apb.set_value('T_fluid_in_1', 'named_expression', Inputs_PyFluent, T_fluid_in0)


e_PV = 2.5e-3
volume_PV = 1134*1708*e_PV*1e-6
Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

# On initialise les a_f et b_f avec le modèle
for i in range(1, nb_hx + 1):
    apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
    apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])



## Initialisation
L_abs = panelSpecs['main']['L_abs']
c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
jg.standard_initialization(tui, 'cd_fc', 0, 0, 0, c)



nb_it = 1000 # Nombre d'itération lancée sur ANSYS



# ## Modification des paramètres dans ANSYS Fluent
# apb.set_value('T_fluid_in_1', 'named_expression', Inputs_PyFluent, T_fluid_in0) # Au cas où
# for part in range(1, nb_hx+1):
#     T_fluid_in = apb.get_value(f'T_fluid_in_{part}', 'named_expression', Inputs_PyFluent)
#     a_f = apb.get_value(f'a_f_{part}', 'named_expression', Inputs_PyFluent)
#     b_f = apb.get_value(f'b_f_{part}', 'named_expression', Inputs_PyFluent)
#     jg.change_named_expression(tui, f'T_fluid_in_{part}', T_fluid_in, 'K')
#     jg.change_named_expression(tui, f'a_f_{part}', a_f, 'm^-1')
#     jg.change_named_expression(tui, f'b_f_{part}', b_f, 'K m^-1')
# T_fluid_out = apb.get_value(f'T_fluid_in_{nb_hx+1}', 'named_expression', Inputs_PyFluent)
# jg.change_named_expression(tui, f'T_fluid_in_{nb_hx+1}', T_fluid_out, 'K')
# jg.change_named_expression(tui, 'Heat_Generation_Rate', Heat_Generation_Rate, 'W/m^3')



## Lancement de la simulation

solver.solution.run_calculation.iterate(number_of_iterations=nb_it)







jg.write_report(tui,'ht',folder_path_case,f'ht_tot_report_uniform_{big_it}')
jg.write_report(tui,'rad_ht',folder_path_case,f'ht_rad_report_uniform_{big_it}')

Qdot_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_uniform_{big_it}.csv'))
Qdot_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_uniform_{big_it}.csv'))

Qdot_conv = pd.DataFrame(columns=['Component', 'conv_ht'])
Qdot_conv['Component'] = Qdot_tot['Component']
Qdot_conv['conv_ht'] = Qdot_tot['ht']-Qdot_rad['rad_ht']

Qdot_conv.to_csv(os.path.join(folder_path_case,f'ht_conv_report_uniform_{big_it}.csv'),sep=';', index=False)

hx_list =  ['hx_bend_mid_air' , 'hx_bend_mid_pv' , 'hx_bend_yd_air' , 'hx_bend_yd_pv' , 'hx_bend_yu_air' ,
    'hx_bend_yu_pv' , 'hx_flat_yd_air', 'hx_flat_yu_air', 'manifold_yu', 'manifold_yd']

## Partie très moche, difficile de faire autrement pour le moment
part1 = ['manifold_yu']
part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
part3 = ['hx_flat_yu_air']
part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
part5 = ['hx_flat_yd_air']
part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
part7 = ['manifold_yd']

part1_top = []
part2_top = []
part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
part4_top = []
part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
part6_top = []
part7_top = []


Areas_top = []
Areas_back = []
for part in panelSpecs['decomp'].keys() :
    if part != 'main':
        Areas_back.append(pr.back_area_tube_conv_and_rad(panelSpecs[part])/4.75)
        Areas_top.append(pr.top_area_tube_contact_PV(panelSpecs[part])/4.75)


## Sauvegarde des données du modèle CFD
phi_top_list = []
phi_tube_list = []
phi_abs_list = []

for i in range(1, nb_hx+3) :
    if i == 3 or i == 5 :
        phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
        phi_top_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum()/Areas_top[i-1])
        phi_abs_list.append(1e-6)
    else :
        phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
        phi_top_list.append(1e-6)
        phi_abs_list.append(1e-6) # Dans le modèle, on a fait en sorte de ne pas avoir besoin de cette valeur

result_CFD = pd.DataFrame(columns=['component', 'phi_top', 'phi_abs', 'phi_tube'])
result_CFD['component'] = [f'part{i}' for i in range(1, nb_hx+3)]
result_CFD['phi_top'] = phi_top_list 
result_CFD['phi_abs'] = phi_abs_list
result_CFD['phi_tube'] = phi_tube_list 



## Envoi des flux au PVT-thermal-performance-model
result_CFD.to_csv(file_path_result_CFD,sep=';', index=False)
Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)

jg.write_data(tui, folder_path_case, f'cas_ref_CFD_uniform')
jg.write_residuals_file(tui, folder_path_case, f'residuals_cas_ref_CFD_uniform')
jg.write_case(tui, folder_path_case, f'cas_ref_CFD_uniform')

In [ ]:
## Calcul du cas de référence méthode AR

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))

panelSpecs = pr.import_geometry(geometry_path)
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))

case = 7

hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
big_it = hyp['big_it']
T_amb = stepConditions[case]['T_amb']
T_fluid_in0 = stepConditions[case]['T_fluid_in0']

# Créer le chemin complet du dossier de résultats
folder_path_case = os.path.join(folder_path, f"cas_ref")

# Vérifier si le dossier n'existe pas déjà
if not os.path.exists(folder_path_case):
    # Créer le dossier s'il n'existe pas
    os.makedirs(folder_path_case)

hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')

file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'

Elapsed_time = pd.DataFrame(columns=['iteration', 'time'])

# File > Application > Start > folder_path_ansys
# Ensuite copier le numéro du server et le modifier ci-dessous
solver_path = folder_path_ansys + '\server\\' + 'server_info-24664.txt'
# Créer une session
solver = pyfluent.connect_to_fluent(server_info_file_name=solver_path)
tui = solver.tui




## Modification des paramètres invariables

# On modifie les composantes de la gravité en fonction de l'inclinaison du panneau
theta = apb.get_value('theta', 'named_expression', Inputs_PyFluent)
jg.change_gravity(tui, theta)

# On initialise la température ambiante
jg.change_named_expression(tui, 'T_amb', T_amb, 'K')

# On initialise le critère de convergence pour les transferts thermiques
convergence_criteria = apb.get_value('convergence_criteria', 'named_expression', Inputs_PyFluent)
jg.change_named_expression(tui, 'convergence_criteria', convergence_criteria, 'W')




## Première initialisation des paramètres dans ANSYS Fluent

for i in range(len(Inputs_PyFluent)):
    named_expression = Inputs_PyFluent.iloc[i]['named_expression']
    value = Inputs_PyFluent.iloc[i]['value']
    unit = Inputs_PyFluent.iloc[i]['unit']
    jg.change_named_expression(tui, named_expression, value, unit)




# Initialisation des valeurs avec PVT-thermal-performances-model
df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)

# On initialise la première température avec le modèle et le transfert radiatif
apb.set_value('T_fluid_in_1', 'named_expression', Inputs_PyFluent, T_fluid_in0)


e_PV = 2.5e-3
volume_PV = 1134*1708*e_PV*1e-6
Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

# On initialise les a_f et b_f avec le modèle
for i in range(1, nb_hx + 1):
    apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
    apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])


# Met à jour les méthodes dans les hypothèses
hyp['method_h_top_g_exchanger'] = 'CFD'
hyp['method_h_back_abs'] = 'CFD'
hyp['method_h_back_tube'] = 'CFD'
hyp['method_h_rad_back_tube'] = 'CFD'



## Initialisation
L_abs = panelSpecs['main']['L_abs']
c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
jg.standard_initialization(tui, 'cd_fc', 0, 0, 0, c)


limit_big_it = 7 # Nombre d'itérations maximum
convergence_criteria = 1e-1 # Critère de convergence pour le transfert thermique
nb_it = 30 # Nombre d'itération lancée sur ANSYS
iteration = 0


while iteration < limit_big_it :
    
    ## Début boucle itération

    time_start = time.time()

    try : 
        ## Calcul des températures

        for i in range(2, nb_hx+2) :
            T_fluid_in = res[f'part{i}']['df_one']['T_fluid_in'].loc[0]
            apb.set_value(f'T_fluid_in_{i-1}', 'named_expression', Inputs_PyFluent, T_fluid_in)
        T_fluid_in_man = res[f'part1']['df_one']['T_fluid_mean'].loc[0]
        T_fluid_out_man = res[f'part7']['df_one']['T_fluid_mean'].loc[0]
        apb.set_value(f'T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in_man)
        apb.set_value(f'T_fluid_out_man', 'named_expression', Inputs_PyFluent, T_fluid_out_man)


        ## Sauvegarde des données du modèle 1D
        # Sauvegarde de df_one
        keys = list(res['part1']['df_one'].keys())
        values_dict = {key: [] for key in keys}

        # Collecte des valeurs dans des listes pour chaque clé
        for i in range(1, nb_hx+3):
            part_key = f'part{i}'
            if part_key in res:
                for key in keys:
                    values_dict[key].append(res[part_key]['df_one'][key].values[0])

        # Création du DataFrame à partir des listes de valeurs
        result_1D_df_one = pd.DataFrame(values_dict)

        # Ajouter les noms des index pour refléter les parties
        result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
        result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

        # Sauvegarde de slices_df
        keys = list(res['part1']['slices_df'].keys())
        values_dict = {key: [] for key in keys}

        # Collecte des valeurs dans des listes pour chaque clé
        for part in range(1, nb_hx+3):
            part_key = f'part{part}'
            if part_key in res:
                for key in keys:
                    values_dict[key].append(res[part_key]['slices_df'][key].values[0])

        # Création du DataFrame à partir des listes de valeurs
        result_1D_slices_df = pd.DataFrame(values_dict)
        result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
        result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)





        ## Modification des paramètres dans ANSYS Fluent
        apb.set_value('T_fluid_in_1', 'named_expression', Inputs_PyFluent, T_fluid_in0) # Au cas où
        for part in range(1, nb_hx+1):
            T_fluid_in = apb.get_value(f'T_fluid_in_{part}', 'named_expression', Inputs_PyFluent)
            a_f = apb.get_value(f'a_f_{part}', 'named_expression', Inputs_PyFluent)
            b_f = apb.get_value(f'b_f_{part}', 'named_expression', Inputs_PyFluent)
            jg.change_named_expression(tui, f'T_fluid_in_{part}', T_fluid_in, 'K')
            jg.change_named_expression(tui, f'a_f_{part}', a_f, 'm^-1')
            jg.change_named_expression(tui, f'b_f_{part}', b_f, 'K m^-1')
        T_fluid_out = apb.get_value(f'T_fluid_in_{nb_hx+1}', 'named_expression', Inputs_PyFluent)
        jg.change_named_expression(tui, f'T_fluid_in_{nb_hx+1}', T_fluid_out, 'K')
        jg.change_named_expression(tui, 'Heat_Generation_Rate', Heat_Generation_Rate, 'W/m^3')



        ## Lancement de la simulation

        solver.solution.run_calculation.iterate(number_of_iterations=nb_it)




        jg.write_report(tui,'ht',folder_path_case,f'ht_tot_report_{big_it}')
        jg.write_report(tui,'rad_ht',folder_path_case,f'ht_rad_report_{big_it}')

        Qdot_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{big_it}.csv'))
        Qdot_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{big_it}.csv'))

        Qdot_conv = pd.DataFrame(columns=['Component', 'conv_ht'])
        Qdot_conv['Component'] = Qdot_tot['Component']
        Qdot_conv['conv_ht'] = Qdot_tot['ht']-Qdot_rad['rad_ht']

        Qdot_conv.to_csv(os.path.join(folder_path_case,f'ht_conv_report_{big_it}.csv'),sep=';', index=False)

        hx_list =  ['hx_bend_mid_air' , 'hx_bend_mid_pv' , 'hx_bend_yd_air' , 'hx_bend_yd_pv' , 'hx_bend_yu_air' ,
            'hx_bend_yu_pv' , 'hx_flat_yd_air', 'hx_flat_yu_air', 'manifold_yu', 'manifold_yd']

        ## Partie très moche, difficile de faire autrement pour le moment
        part1 = ['manifold_yu']
        part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
        part3 = ['hx_flat_yu_air']
        part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
        part5 = ['hx_flat_yd_air']
        part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
        part7 = ['manifold_yd']

        part1_top = []
        part2_top = []
        part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
        part4_top = []
        part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
        part6_top = []
        part7_top = []


        Areas_top = []
        Areas_back = []
        for part in panelSpecs['decomp'].keys() :
            if part != 'main':
                Areas_back.append(pr.back_area_tube_conv_and_rad(panelSpecs[part])/4.75)
                Areas_top.append(pr.top_area_tube_contact_PV(panelSpecs[part])/4.75)


        ## Sauvegarde des données du modèle CFD
        phi_top_list = []
        phi_tube_list = []
        phi_abs_list = []

        for i in range(1, nb_hx+3) :
            if i == 3 or i == 5 :
                phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                phi_top_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum()/Areas_top[i-1])
                phi_abs_list.append(1e-6)
            else :
                phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                phi_top_list.append(1e-6)
                phi_abs_list.append(1e-6) # Dans le modèle, on a fait en sorte de ne pas avoir besoin de cette valeur

        result_CFD = pd.DataFrame(columns=['component', 'phi_top', 'phi_abs', 'phi_tube'])
        result_CFD['component'] = [f'part{i}' for i in range(1, nb_hx+3)]
        result_CFD['phi_top'] = phi_top_list 
        result_CFD['phi_abs'] = phi_abs_list
        result_CFD['phi_tube'] = phi_tube_list 



        ## Envoi des flux au PVT-thermal-performance-model
        result_CFD.to_csv(file_path_result_CFD,sep=';', index=False)
        Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)


        # Pour permettre au modèle de ne pas recommencer depuis le début
        hyp['specific_init'] = df_one['T_PV'].values[0]

        ## Appel du PVT-thermal-performances-model
        df_one,res = ty.simu_one_steady_state_all_he(panelSpecs,stepConditions[case],hyp)
        # stepConditions[case] = save_stepConditions.copy() ## On remet les conditions initiales car simu_one_steady_state_all_he modifie stepConditions

        Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
        apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

        # On initialise les a_f et b_f avec le modèle
        for i in range(1, nb_hx + 1):
            apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
            apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])



        ## On incrémente 

        big_it = big_it + 1
        hyp['big_it'] = big_it
        file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
        file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
        file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
        file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'


        print(f'Cas {case+1} iteration {big_it} / {limit_big_it} terminée')
        iteration += 1

    except Exception as e:
        big_it = big_it + 1
        print(f'Cas {case} iteration {big_it} / {limit_big_it} non aboutie : {e}')
        iteration = limit_big_it
        continue

    jg.write_data(tui, folder_path_case, f'cas{case+1}_it{big_it}')
    jg.write_residuals_file(tui, folder_path_case, f'residuals_cas{case+1}_it{big_it}')
    jg.write_case(tui, folder_path_case, f'cas{case+1}_it{big_it}')

    time_end = time.time()
    Elapsed_time.loc[iteration] = {'iteration': big_it, 'time': time_end - time_start}

    # ## Fin boucle itération

Elapsed_time.to_csv(os.path.join(folder_path_case,f'Elapsed_time_cas_{case+1}.csv'),sep=';', index=False)


## Dernière sauvegarde des données du modèle 1D
# Sauvegarde de df_one
keys = list(res['part1']['df_one'].keys())
values_dict = {key: [] for key in keys}

# Collecte des valeurs dans des listes pour chaque clé
for i in range(1, nb_hx+3):
    part_key = f'part{i}'
    if part_key in res:
        for key in keys:
            values_dict[key].append(res[part_key]['df_one'][key].values[0])

# Création du DataFrame à partir des listes de valeurs
result_1D_df_one = pd.DataFrame(values_dict)

# Ajouter les noms des index pour refléter les parties
result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

# Sauvegarde de slices_df
keys = list(res['part1']['slices_df'].keys())
values_dict = {key: [] for key in keys}

# Collecte des valeurs dans des listes pour chaque clé
for part in range(1, nb_hx+3):
    part_key = f'part{part}'
    if part_key in res:
        for key in keys:
            values_dict[key].append(res[part_key]['slices_df'][key].values[0])

# Création du DataFrame à partir des listes de valeurs
result_1D_slices_df = pd.DataFrame(values_dict)
result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)


### Géométries différentes

In [ ]:
## Notebook qui fait la boucle sur les géométries et les cas

folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
folder_path_ansys = os.path.abspath(os.path.join(folder_path, '..'))


steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')

Inputs_PyFluent = pd.read_excel(folder_path + '\\' + 'Inputs_PyFluent.xlsx')

geometries_dict = pd.read_excel(folder_path + '\\' + 'test-geometries.xlsx') ## Contient les chemins des géométries et les noms des fichiers

nb_hx = int(apb.get_value('nb_hx', 'named_expression', Inputs_PyFluent))
# nb_case = len(stepConditions)
nb_case = 5
nb_geom = len(geometries_dict)

# File > Application > Start > folder_path_ansys
# Ensuite copier le numéro du server et le modifier ci-dessous
solver_path = folder_path_ansys + '\server\\' + 'server_info-27608.txt'
# Créer une session
solver = pyfluent.connect_to_fluent(server_info_file_name=solver_path)
tui = solver.tui


for geom in range(nb_geom) :

    geom_path = geometries_dict['mesh_path'].iloc[geom]
    geom_name = geometries_dict['mesh_name_wo_ext'].iloc[geom]
    geom_panel_name = geometries_dict['mesh_panel'].iloc[geom]

    geometry_path = folder_path + '\\' + f'Inputs_TEST_{geom_panel_name}.xlsx'
    panelSpecs = pr.import_geometry(geometry_path)

    folder_path_geom = os.path.join(folder_path, geom_name)
    # Vérifier si le dossier n'existe pas déjà
    if not os.path.exists(folder_path_case):
        # Créer le dossier s'il n'existe pas
        os.makedirs(folder_path_geom)

    jg.change_mesh(tui, geom_path, geom_name)
    jg.create_radiation(tui, geom_name)

    for case in range(nb_case) :
    # case = 7
    # if True :

        ## Début de la boucle for case

        hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
        big_it = hyp['big_it']
        T_amb = stepConditions[case]['T_amb']
        T_fluid_in0 = stepConditions[case]['T_fluid_in0']
        # save_stepConditions = stepConditions[case].copy() ## Modification directement dans steady_one_steady_state_all_he

        # Créer le chemin complet du dossier de résultats
        folder_path_case = os.path.join(folder_path_geom, f"cas_{case}")

        # Vérifier si le dossier n'existe pas déjà
        if not os.path.exists(folder_path_case):
            # Créer le dossier s'il n'existe pas
            os.makedirs(folder_path_case)

        hyp['CFD_ht_path'] = os.path.join(folder_path_case, 'test')

        file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
        file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
        file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
        file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'

        Elapsed_time = pd.DataFrame(columns=['iteration', 'time'])


        ## Modification des paramètres invariables

        # On modifie les composantes de la gravité en fonction de l'inclinaison du panneau
        theta = apb.get_value('theta', 'named_expression', Inputs_PyFluent)
        jg.change_gravity(tui, theta)

        # On initialise la température ambiante
        jg.change_named_expression(tui, 'T_amb', T_amb, 'K')

        # On initialise le critère de convergence pour les transferts thermiques
        convergence_criteria = apb.get_value('convergence_criteria', 'named_expression', Inputs_PyFluent)
        jg.change_named_expression(tui, 'convergence_criteria', convergence_criteria, 'W')




        ## Première initialisation des paramètres dans ANSYS Fluent

        for i in range(len(Inputs_PyFluent)):
            named_expression = Inputs_PyFluent.iloc[i]['named_expression']
            value = Inputs_PyFluent.iloc[i]['value']
            unit = Inputs_PyFluent.iloc[i]['unit']
            jg.change_named_expression(tui, named_expression, value, unit)




        # Initialisation des valeurs avec PVT-thermal-performances-model
        df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, stepConditions[case], hyp)
        # stepConditions[case] = save_stepConditions.copy()

        # On initialise la première température avec le modèle et le transfert radiatif
        apb.set_value('T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in0)


        e_PV = 2.5e-3
        volume_PV = 1134*1708*e_PV*1e-6
        Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
        apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

        # On initialise les a_f et b_f avec le modèle
        for i in range(1, nb_hx + 1):
            apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
            apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])
        

        # Met à jour les méthodes dans les hypothèses
        hyp['method_h_top_g_exchanger'] = 'CFD'
        hyp['method_h_back_abs'] = 'CFD'
        hyp['method_h_back_tube'] = 'CFD'
        hyp['method_h_rad_back_tube'] = 'CFD'



        ## Initialisation
        L_abs = panelSpecs['main']['L_abs']
        c = bht.speed_natural_convection(T_fluid_in0, T_amb, theta,L_abs)
        jg.standard_initialization(tui, 'cd_fc', 0, 0, 0, c)


        limit_big_it = 7 # Nombre d'itérations maximum
        convergence_criteria = 1e-1 # Critère de convergence pour le transfert thermique
        nb_it = 30 # Nombre d'itération lancée sur ANSYS
        iteration = 0


        while iteration < limit_big_it :
            
            ## Début boucle itération

            time_start = time.time()

            # try : 
            ## Calcul des températures

            # apb.fill_initial_temperature_dict(Inputs_PyFluent)

            for i in range(2, nb_hx+2) :
                T_fluid_in = res[f'part{i}']['df_one']['T_fluid_in'].loc[0]
                apb.set_value(f'T_fluid_in_{i-1}', 'named_expression', Inputs_PyFluent, T_fluid_in)
            T_fluid_in_man = res[f'part1']['df_one']['T_fluid_mean'].loc[0]
            T_fluid_out_man = res[f'part7']['df_one']['T_fluid_mean'].loc[0]
            apb.set_value(f'T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in_man)
            apb.set_value(f'T_fluid_out_man', 'named_expression', Inputs_PyFluent, T_fluid_out_man)


            ## Sauvegarde des données du modèle 1D
            # Sauvegarde de df_one
            keys = list(res['part1']['df_one'].keys())
            values_dict = {key: [] for key in keys}

            # Collecte des valeurs dans des listes pour chaque clé
            for i in range(1, nb_hx+3):
                part_key = f'part{i}'
                if part_key in res:
                    for key in keys:
                        values_dict[key].append(res[part_key]['df_one'][key].values[0])

            # Création du DataFrame à partir des listes de valeurs
            result_1D_df_one = pd.DataFrame(values_dict)

            # Ajouter les noms des index pour refléter les parties
            result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
            result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

            # Sauvegarde de slices_df
            keys = list(res['part1']['slices_df'].keys())
            values_dict = {key: [] for key in keys}

            # Collecte des valeurs dans des listes pour chaque clé
            for part in range(1, nb_hx+3):
                part_key = f'part{part}'
                if part_key in res:
                    for key in keys:
                        values_dict[key].append(res[part_key]['slices_df'][key].values[0])

            # Création du DataFrame à partir des listes de valeurs
            result_1D_slices_df = pd.DataFrame(values_dict)
            result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
            result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)





            ## Modification des paramètres dans ANSYS Fluent
            apb.set_value('T_fluid_in_man', 'named_expression', Inputs_PyFluent, T_fluid_in0) # Au cas où
            for part in range(1, nb_hx+1):
                T_fluid_in = apb.get_value(f'T_fluid_in_{part}', 'named_expression', Inputs_PyFluent)
                a_f = apb.get_value(f'a_f_{part}', 'named_expression', Inputs_PyFluent)
                b_f = apb.get_value(f'b_f_{part}', 'named_expression', Inputs_PyFluent)
                jg.change_named_expression(tui, f'T_fluid_in_{part}', T_fluid_in, 'K')
                jg.change_named_expression(tui, f'a_f_{part}', a_f, 'm^-1')
                jg.change_named_expression(tui, f'b_f_{part}', b_f, 'K m^-1')
            jg.change_named_expression(tui, 'Heat_Generation_Rate', Heat_Generation_Rate, 'W/m^3')
            jg.change_named_expression(tui, 'T_fluid_in_man',T_fluid_in_man, 'K')
            jg.change_named_expression(tui, 'T_fluid_out_man', T_fluid_out_man, 'K')



            ## Lancement de la simulation

            solver.solution.run_calculation.iterate(number_of_iterations=nb_it)




            jg.write_report(tui,'ht',folder_path_case,f'ht_tot_report_{big_it}')
            jg.write_report(tui,'rad_ht',folder_path_case,f'ht_rad_report_{big_it}')

            Qdot_tot = pd.read_csv(os.path.join(folder_path_case,f'ht_tot_report_{big_it}.csv'))
            Qdot_rad = pd.read_csv(os.path.join(folder_path_case,f'ht_rad_report_{big_it}.csv'))

            Qdot_conv = pd.DataFrame(columns=['Component', 'conv_ht'])
            Qdot_conv['Component'] = Qdot_tot['Component']
            Qdot_conv['conv_ht'] = Qdot_tot['ht']-Qdot_rad['rad_ht']

            Qdot_conv.to_csv(os.path.join(folder_path_case,f'ht_conv_report_{big_it}.csv'),sep=';', index=False)

            hx_list =  ['hx_bend_mid_air' , 'hx_bend_mid_pv' , 'hx_bend_yd_air' , 'hx_bend_yd_pv' , 'hx_bend_yu_air' ,
                'hx_bend_yu_pv' , 'hx_flat_yd_air', 'hx_flat_yu_air', 'manifold_yu', 'manifold_yd']

            ## Partie très moche, difficile de faire autrement pour le moment
            part1 = ['manifold_yu']
            part2 = ['hx_bend_yu_air', 'hx_bend_yu_pv']
            part3 = ['hx_flat_yu_air']
            part4 = ['hx_bend_mid_air', 'hx_bend_mid_pv']
            part5 = ['hx_flat_yd_air']
            part6 = ['hx_bend_yd_air', 'hx_bend_yd_pv']
            part7 = ['manifold_yd']

            part1_top = []
            part2_top = []
            part3_top = ['hx_flat_yu_pv-pv_backsheet-cd-cd1-pv-corps']
            part4_top = []
            part5_top = ['hx_flat_yd_pv-pv_backsheet-cd-cd1-pv-corps']
            part6_top = []
            part7_top = []


            Areas_top = []
            Areas_back = []
            for part in panelSpecs['decomp'].keys() :
                if part != 'main':
                    Areas_back.append(pr.back_area_tube_conv_and_rad(panelSpecs[part])/4.75)
                    Areas_top.append(pr.top_area_tube_contact_PV(panelSpecs[part])/4.75)


            ## Sauvegarde des données du modèle CFD
            phi_top_list = []
            phi_tube_list = []
            phi_abs_list = []

            for i in range(1, nb_hx+3) :
                if i == 3 or i == 5 :
                    phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                    phi_top_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}_top'))]['conv_ht'].sum()/Areas_top[i-1])
                    phi_abs_list.append(1e-6)
                else :
                    phi_tube_list.append(Qdot_conv[Qdot_conv['Component'].isin(eval(f'part{i}'))]['conv_ht'].sum()/Areas_back[i-1])
                    phi_top_list.append(1e-6)
                    phi_abs_list.append(1e-6) # Dans le modèle, on a fait en sorte de ne pas avoir besoin de cette valeur

            result_CFD = pd.DataFrame(columns=['component', 'phi_top', 'phi_abs', 'phi_tube'])
            result_CFD['component'] = [f'part{i}' for i in range(1, nb_hx+3)]
            result_CFD['phi_top'] = phi_top_list 
            result_CFD['phi_abs'] = phi_abs_list
            result_CFD['phi_tube'] = phi_tube_list 



            ## Envoi des flux au PVT-thermal-performance-model
            result_CFD.to_csv(file_path_result_CFD,sep=';', index=False)
            Inputs_PyFluent.to_csv(file_path_result_PyFluent,sep=';', index=False)


            # Pour permettre au modèle de ne pas recommencer depuis le début
            hyp['specific_init'] = df_one['T_PV'].values[0]

            ## Appel du PVT-thermal-performances-model
            df_one,res = ty.simu_one_steady_state_all_he(panelSpecs,stepConditions[case],hyp)
            # stepConditions[case] = save_stepConditions.copy() ## On remet les conditions initiales car simu_one_steady_state_all_he modifie stepConditions

            Heat_Generation_Rate = -df_one['Qdot_PV_sky'].values[0]/volume_PV
            apb.set_value('Heat_Generation_Rate', 'named_expression', Inputs_PyFluent, Heat_Generation_Rate)

            # On initialise les a_f et b_f avec le modèle
            for i in range(1, nb_hx + 1):
                apb.set_value(f'a_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['a_f'].values[0])
                apb.set_value(f'b_f_{i}', 'named_expression', Inputs_PyFluent, res[f'part{i}']['slices_df']['b_f'].values[0])



            ## On incrémente 

            big_it = big_it + 1
            hyp['big_it'] = big_it
            file_path_result_CFD = hyp['CFD_ht_path']+f'_{big_it}.csv' ## On ne peut pas l'appeler autrement pour le moment, il y a un appel dans la fonction simu_on_steadyState
            file_path_result_slices_df = hyp['CFD_ht_path']+'_slices_df' + f'_{big_it}.csv'
            file_path_result_df_one = hyp['CFD_ht_path']+'_df_one' + f'_{big_it}.csv'
            file_path_result_PyFluent = hyp['CFD_ht_path']+'_PyFluent' + f'_{big_it}.csv'


            print(f'Cas {case} iteration {big_it} / {limit_big_it} terminée')
            iteration += 1

            # except Exception as e:
            #     big_it = big_it + 1
            #     print(f'Cas {case} iteration {big_it} / {limit_big_it} non aboutie : {e}')
            #     iteration = limit_big_it
            #     continue

            jg.write_data(tui, folder_path_case, f'cas{case}_it{big_it}')
            jg.write_residuals_file(tui, folder_path_case, f'residuals_cas{case}_it{big_it}')
            jg.write_case(tui, folder_path_case, f'cas{case}_it{big_it}')

            time_end = time.time()
            Elapsed_time.loc[iteration] = {'iteration': big_it, 'time': time_end - time_start}

            # ## Fin boucle itération

        Elapsed_time.to_csv(os.path.join(folder_path_case,f'Elapsed_time_cas_{case}.csv'),sep=';', index=False)

    ## Fin de la boucle for case


    ## Dernière sauvegarde des données du modèle 1D
    # Sauvegarde de df_one
    keys = list(res['part1']['df_one'].keys())
    values_dict = {key: [] for key in keys}

    # Collecte des valeurs dans des listes pour chaque clé
    for i in range(1, nb_hx+3):
        part_key = f'part{i}'
        if part_key in res:
            for key in keys:
                values_dict[key].append(res[part_key]['df_one'][key].values[0])

    # Création du DataFrame à partir des listes de valeurs
    result_1D_df_one = pd.DataFrame(values_dict)

    # Ajouter les noms des index pour refléter les parties
    result_1D_df_one.index = [f'part{i}' for i in range(1, nb_hx + 3)]
    result_1D_df_one.to_csv(file_path_result_df_one,sep=';', index=False)

    # Sauvegarde de slices_df
    keys = list(res['part1']['slices_df'].keys())
    values_dict = {key: [] for key in keys}

    # Collecte des valeurs dans des listes pour chaque clé
    for part in range(1, nb_hx+3):
        part_key = f'part{part}'
        if part_key in res:
            for key in keys:
                values_dict[key].append(res[part_key]['slices_df'][key].values[0])

    # Création du DataFrame à partir des listes de valeurs
    result_1D_slices_df = pd.DataFrame(values_dict)
    result_1D_slices_df.index = [f'part{i}' for i in range(1, nb_hx+3)]
    result_1D_slices_df.to_csv(file_path_result_slices_df,sep=';', index=False)
